# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `pub_paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [60]:
!cat publications.tsv

pub_date	status	title	venue	excerpt	citation	url_slug	pub_paper_url	wp_paper_url	slides_url	opt1_text	opt1_url	opt2_text	opt2_url	opt3_text	opt3_url
2023-11-01	in_progress	What Do We Learn From Reading Every FOMC Transcript?	Working Paper		"Coibion, Olivier, Marc Dordal i Carreras, Yuriy Gorodnichenko, and Cooper Howes. ""What Do We Learn From Reading Every FOMC Transcript?""  <i>Working Paper</i> (2023)."	CDGH_FOMC_transcript									
2023-11-01	resting_work	The Spatial Transmission of US Banking Panics	Working Paper		"Dordal i Carreras, Marc and Seung Joo Lee. ""The Spatial Transmission of US Banking Panics.""  <i>Working Paper</i> (2023)."	DJ_Bank_Panics									
2023-11-01	in_progress	"Gender Gap, Structural Change and Female Comparative Advantage: A Quantitative Analysis of China"	Working Paper	"This paper presents a theoretical framework to reconcile the declining female labor force participation (FLFP) rate and the diverging gender gap in workforce participation in China with th

## Import pandas

We are using the very handy pandas library for dataframes.

In [61]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [62]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,status,title,venue,excerpt,citation,url_slug,pub_paper_url,wp_paper_url,slides_url,opt1_text,opt1_url,opt2_text,opt2_url,opt3_text,opt3_url
0,2023-11-01,in_progress,What Do We Learn From Reading Every FOMC Trans...,Working Paper,NaN,"Coibion, Olivier, Marc Dordal i Carreras, Yuri...",CDGH_FOMC_transcript,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-01,resting_work,The Spatial Transmission of US Banking Panics,Working Paper,NaN,"Dordal i Carreras, Marc and Seung Joo Lee. ""Th...",DJ_Bank_Panics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11-01,in_progress,"Gender Gap, Structural Change and Female Compa...",Working Paper,This paper presents a theoretical framework to...,"Dordal i Carreras, Cassie Xiang. ""Gender Gap, ...",DX_Gender_Gap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-11-01,in_progress,Empirical Estimation of Bond Market Segmentation,Working Paper,NaN,"Carruthers, Anna, Marc Dordal i Carreras, and ...",CDJ_Market_segmentation_empiric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-11-01,in_progress,A New Indeterminacy with Fluctuations in Volat...,Working Paper,NaN,"Dordal i Carreras, Marc and Seung Joo Lee. ""A ...",DJ_Indeterminacy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-11-01,in_progress,A Higher-Order Forward Guidance,Working Paper,NaN,"Dordal i Carreras, Marc and Seung Joo Lee. ""A ...",DJ_Higher_OrderFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-11-01,working_paper,A Theory of Keynesian Demand and Supply Intera...,Working Paper,NaN,"Dordal i Carreras, Marc, Seung Joo Lee, and Zh...",DJQ_ASAD_revisited,NaN,http://marcdordal.github.io/files/WP_AD_AS.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2023-11-01,working_paper,A Unified Theory of the Term-Structure and Mon...,Working Paper,We develop a New-Keynesian framework that inco...,"Joo Lee, Seung, and Marc Dordal i Carreras. ""A...",JD_term_structure_theory,NaN,http://marcdordal.github.io/files/WP_Term_Stru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2023-11-01,working_paper,"Self-fulfilling Volatility, Risk-Premium, and ...",Working Paper,We develop a New-Keynesian framework with stoc...,"Joo Lee, Seung, and Marc Dordal i Carreras. ""S...",JD_sunspot,NaN,http://marcdordal.github.io/files/WP_Sunspot_r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2023-11-01,working_paper,"Efficiency, Risk and the Gains from Trade in I...",Working Paper,We propose a model of the financial sector tha...,"Dordal i Carreras, Marc, Matthias Hoelzlein, a...",DHO_Interbank,NaN,http://marcdordal.github.io/files/WP_Trade_Mod...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [63]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [64]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: published_papers"""
    
    md += """\npermalink: /published_paper/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.pub_paper_url)) > 5:
        md += "\npubpaperurl: '" + item.pub_paper_url + "'"
    
    if len(str(item.wp_paper_url)) > 5:
        md += "\nwppaperurl: '" + item.wp_paper_url + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"
     
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.pub_paper_url)) > 5:
        md += "\n[Access published version here](" + item.pub_paper_url + ")\n" 
    
    if len(str(item.wp_paper_url)) > 5:
        md += "\n[Download working paper here](" + item.wp_paper_url + ")\n" 

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 
    
    if len(str(item.opt1_url)) > 5:
        md += "\n[" + item.opt1_text + "](" + item.opt1_url + ")\n"     
    
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
    
    
    ## Save markdown file into corresponding GitHub folder
    if item.status == "published_paper":
        with open("../_published_papers/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "working_paper":
        with open("../_working_papers/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "policy_paper":
        with open("../_policy_papers/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "in_progress":
        with open("../_works_in_progress/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "resting_work":
        with open("../_resting_works/" + md_filename, 'w') as f:
            f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [65]:
!ls ../_published_papers/

2016-11-01-DCG_Infreq_ZLB.md


In [66]:
!cat ../_published_papers/2016-11-01-DCG_Infreq_ZLB.md

---
title: "Infrequent but long-lived zero lower bound episodes and the optimal rate of inflation"
collection: published_papers
permalink: /published_paper/2016-11-01-DCG_Infreq_ZLB
excerpt: 'Countries rarely hit the zero lower bound (ZLB) on interest rates, but when they do, these episodes tend to be very long-lived. These two features are difficult to incorporate jointly into macroeconomic models using typical representations of shock processes. We introduce a regime-switching representation of risk premium shocks into an otherwise standard New Keynesian model to generate a realistic distribution of ZLB durations. We discuss what different calibrations of this model imply for optimal inflation rates.'
date: 2016-11-01
venue: 'Annual Review of Economics'
pubpaperurl: 'https://www.annualreviews.org/doi/abs/10.1146/annurev-economics-080315-015306'
wppaperurl: 'http://marcdordal.github.io/files/WP_Infreq_ZLB.pdf'
slidesurl: 'http://marcdordal.github.io/files/slides_Infreq_ZLB.pdf'
citati